## EDA


In [1]:
#pip install scikit-learn

In [2]:
#pip install seaborn

In [3]:
#pip install h2o

In [4]:
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
import h2o
from h2o.automl import H2OAutoML


In [5]:


# Read the CSV file into a DataFrame
df = pd.read_csv("/Users/dezmon/Documents/AI in Healthcare/LOS_MIMIC/merged_3_files.csv")

# Display the DataFrame
df

,Age,gender,ethnicity,insurance,id,label,R78,I10,I82,L53,...,D16,S69,C03,N75,X79,F18,Z67,E25,W67,H62
0,55,F,WHITE,Other,30327812,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,46,F,WHITE,Other,37542711,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,68,F,BLACK/AFRICAN AMERICAN,Medicare,31832304,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,53,F,OTHER,Medicare,37106138,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,80,F,WHITE,Other,34599502,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56332,71,F,BLACK/AFRICAN AMERICAN,Medicare,32553956,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
56333,42,M,OTHER,Other,35840543,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56334,41,M,WHITE,Medicaid,37569254,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56335,58,M,WHITE,Other,39944651,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:

df.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,56337.0,6.369753e+01,1.647302e+01,18.0,54.0,65.0,76.0,91.0
id,56337.0,3.500444e+07,2.876853e+06,30000010.0,32527524.0,35004476.0,37492660.0,39998706.0
label,56337.0,1.734384e-01,3.786292e-01,0.0,0.0,0.0,0.0,1.0
R78,56337.0,7.231482e-02,2.590108e-01,0.0,0.0,0.0,0.0,1.0
I10,56337.0,3.792179e-01,4.851967e-01,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...
F18,56337.0,1.775032e-05,4.213113e-03,0.0,0.0,0.0,0.0,1.0
Z67,56337.0,1.775032e-05,4.213113e-03,0.0,0.0,0.0,0.0,1.0
E25,56337.0,1.775032e-05,4.213113e-03,0.0,0.0,0.0,0.0,1.0
W67,56337.0,1.775032e-05,4.213113e-03,0.0,0.0,0.0,0.0,1.0


In [7]:
## Removing columns with 80% missing values

threshold = 0.80  # Set your desired threshold (80% in this case)
df = df[df.columns[df.isnull().mean() < threshold]]

In [8]:
null_counts = df.isnull().sum()
null_columns = null_counts[null_counts > 0]
print(null_counts)

Age          0
gender       0
ethnicity    0
insurance    0
id           0
            ..
F18          0
Z67          0
E25          0
W67          0
H62          0
Length: 1470, dtype: int64


In [9]:
from scipy import stats
#Outler Detection 
threshold = 3  #adjust this threshold based on your data

# Create a new DataFrame to store data without outliers
df_no_outliers = df.copy()

# Loop through the columns and remove outliers for each column
for column_name in df.columns:
    if df[column_name].dtype in [int, float]:
        z_scores = stats.zscore(df[column_name])
        df_no_outliers = df_no_outliers[abs(z_scores) <= threshold]


/var/folders/l1/drp5zdgd1cb8tf8xkmzh4zrh0000gn/T/ipykernel_63014/4262240421.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_no_outliers = df_no_outliers[abs(z_scores) <= threshold]
/var/folders/l1/drp5zdgd1cb8tf8xkmzh4zrh0000gn/T/ipykernel_63014/4262240421.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_no_outliers = df_no_outliers[abs(z_scores) <= threshold]
/var/folders/l1/drp5zdgd1cb8tf8xkmzh4zrh0000gn/T/ipykernel_63014/4262240421.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_no_outliers = df_no_outliers[abs(z_scores) <= threshold]
/var/folders/l1/drp5zdgd1cb8tf8xkmzh4zrh0000gn/T/ipykernel_63014/4262240421.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_no_outliers = df_no_outliers[abs(z_scores) <= threshold]
/var/folders/l1/drp5zdgd1cb8tf8xkmzh4zrh0000gn/T/ipykernel_63014/4262240421.py:12: UserWarning: Boolean Series key w

In [10]:
df_no_outliers.describe

<bound method NDFrame.describe of        Age gender ethnicity insurance        id  label  R78  I10  I82  L53  \
13      50      M     WHITE     Other  30922999      0    0    0    0    0   
42      45      M     WHITE     Other  30741821      0    0    0    0    0   
44      61      F     WHITE     Other  30272829      0    0    0    0    0   
73      88      M     WHITE     Other  33871042      0    0    0    0    0   
79      40      M     WHITE  Medicare  31631748      0    0    0    0    0   
...    ...    ...       ...       ...       ...    ...  ...  ...  ...  ...   
56307   66      M     WHITE  Medicare  34775055      0    0    1    0    0   
56327   52      F     WHITE  Medicare  33754047      1    0    0    0    0   
56328   52      F     WHITE  Medicare  36028546      1    0    0    0    0   
56333   42      M     OTHER     Other  35840543      1    0    0    0    0   
56335   58      M     WHITE     Other  39944651      0    0    0    0    0   

       ...  D16  S69  C03  N7

# RFE Feature Selection

In [11]:


# Separate your features and target variable
X = df_no_outliers.drop('id', axis=1)  # X contains the features
X = df_no_outliers.drop('label', axis=1)  # X contains the features
y = df_no_outliers['label']  # y is the target variable

# Convert categorical columns to numerical using one-hot encoding
X = pd.get_dummies(X)  # This will convert categorical columns to one-hot encoded columns

# Create an instance of the model you want to use for RFE
model = LinearRegression()

# Initialize RFE with the model and the number of features to select
num_features_to_select = 50  # You can set the number of features you want to select
rfe = RFE(model, n_features_to_select=num_features_to_select)

# Fit RFE to the data
rfe.fit(X, y)

# Get the selected features
selected_features = X.columns[rfe.support_]

print("Selected Features:")
print(selected_features)

Selected Features:
Index(['Age', 'id', 'R78', 'I10', 'E87', 'N39', 'N17', 'E78', 'J98', 'R00',
       'I95', 'Z85', 'J18', 'G47', 'Z91', 'Z51', 'D69', 'I47', 'I50', 'J44',
       'D64', 'E11', 'I12', 'N18', 'K22', 'E66', 'Z95', 'G93', 'F41', 'Z68',
       'F32', 'I25', 'R68', 'F10', 'Z87', 'Z79', 'I48', 'gender_F', 'gender_M',
       'ethnicity_AMERICAN INDIAN/ALASKA NATIVE', 'ethnicity_ASIAN',
       'ethnicity_BLACK/AFRICAN AMERICAN', 'ethnicity_HISPANIC/LATINO',
       'ethnicity_OTHER', 'ethnicity_UNABLE TO OBTAIN', 'ethnicity_UNKNOWN',
       'ethnicity_WHITE', 'insurance_Medicaid', 'insurance_Medicare',
       'insurance_Other'],
      dtype='object')


In [12]:
# Create a new dataset with only the selected features
X_selected = X[selected_features]


In [13]:
#pip install --upgrade h2o

In [14]:
selected_X = X[selected_features]

# Convert the target variable to categorical
y = y.astype('category')

# Initialize H2O
h2o.init()

# Convert selected features and target variable to H2OFrame
h2o_selected_X = h2o.H2OFrame(pd.concat([selected_X, y], axis=1))

# Run AutoML
aml = H2OAutoML(max_models=10, seed=42)
aml.train(y='label', training_frame=h2o_selected_X)

# View the AutoML leaderboard
lb = aml.leaderboard
print(lb)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...


CalledProcessError: Command '['/usr/bin/java', '-version']' returned non-zero exit status 1.

In [ ]:
# Select the top model
top_model = aml.leader

# Retrieve details of the top model
model_details = h2o.get_model(top_model.model_id)

# View the top model details
print(model_details)

In [ ]:
# Retrieve variable importance of the top model
feature_importance = top_model.varimp()

# Create lists of feature names and their respective importance values
feature_names = [item[0] for item in feature_importance]
importance_values = [item[2] for item in feature_importance]

# Plotting feature importance
plt.figure(figsize=(10, 6))
plt.barh(feature_names, importance_values, color='skyblue')
plt.xlabel('Importance')
plt.title('Feature Importance of Top H2O Model')
plt.gca().invert_yaxis()  # Invert y-axis for better visualization
plt.show()